## Obtendo os arquivos xml e realizando o parse para o Banco de Dados

In [1]:
import downloadfilesfromTCE as tce
from pymongo import MongoClient
import os


In [2]:
client = MongoClient('localhost',27017)
db = client['db_data_gov']

def save_mongo(dataframe, collection_name=None):
    result = db[collection_name].insert_many(dataframe.to_dict(orient='records'))
    return result

def find_cdIBGE(nmMunicipio = []):
    query = {'municipio' : { '$in': nmMunicipio } }
    projection = { '_id' : 0, 'cdIBGE' : 1}
    result = db.municipio.find(query, projection)
    return result
    

### 1) Obtendo os dados do site Transparencia Para Totos

In [3]:
anos = ['2013', '2014', '2015', '2016', '2017']
municipios = [
    'Londrina'
]

In [4]:
def obterCodigoMunicipio(nomesMunicipios = []) :
    queryResult = find_cdIBGE(municipios)
    m = map(lambda x : x['cdIBGE'], queryResult)
    return list(m)


In [5]:
codMunicipios = obterCodigoMunicipio(municipios)

for ano in anos :
    for cod in codMunicipios :
#         tce.get_file(cod,ano,'licitacao')
        print(cod,ano,'licitacao')

4113700 2013 licitacao
4113700 2013 licitacao
4113700 2014 licitacao
4113700 2014 licitacao
4113700 2015 licitacao
4113700 2015 licitacao
4113700 2016 licitacao
4113700 2016 licitacao
4113700 2017 licitacao
4113700 2017 licitacao


In [6]:
tce.extract_zip_to('licitacao')

extracting... Licitacao_2013_411370.zip
extracting... Licitacao_2013_411990.zip
extracting... Licitacao_2014_411370.zip
extracting... Licitacao_2014_411990.zip
extracting... Licitacao_2015_411370.zip
extracting... Licitacao_2015_411990.zip
extracting... Licitacao_2016_411370.zip
extracting... Licitacao_2016_411990.zip
extracting... Licitacao_2017_411370.zip
extracting... Licitacao_2017_411990.zip


In [6]:
diretorio = "data/licitacao/"
files = os.listdir(diretorio)
files = list(map(lambda f : diretorio + f, files))

len(files)

30

### 2) Parse dos dado e salvando no MongoDB

['Licitacao.xml', 'LicitacaoVencedor.xml', 'LicitacaoParticipante.xml']

In [13]:
files[20]

'data/licitacao/2016_411370_LicitacaoVencedor.xml'

xmlParseCharRef: invalid xmlChar value 28, line 1, column 10148128 (data/licitacao/2015_411370_LicitacaoVencedor.xml, line 1)
XMLSyntaxError occurred  -> using fromXMLAsStringToDataFrame function to process file: 
data/licitacao/2015_411370_LicitacaoVencedor.xml
xmlParseCharRef: invalid xmlChar value 31, line 1, column 6494951 (data/licitacao/2017_411990_LicitacaoVencedor.xml, line 1)
XMLSyntaxError occurred  -> using fromXMLAsStringToDataFrame function to process file: 
data/licitacao/2017_411990_LicitacaoVencedor.xml


True